In [8]:
import pandas as pd

In [9]:
pd.read_csv('train_mod.csv', delim_whitespace=True)

EmptyDataError: No columns to parse from file

In [7]:
test=pd.read_csv('test_mod.csv')

EmptyDataError: No columns to parse from file

In [5]:
train.shape , test.shape

((300000, 115), (200000, 114))

In [23]:
(train.isnull().sum()>0).sum()

3

In [16]:
l = ['C_COUNT_L3_MON_01','C_COUNT_L3_MON_02','C_COUNT_L3_MON_03','C_COUNT_L3_MON_04','C_COUNT_L3_MON_05','C_COUNT_L3_MON_06']

In [17]:
for x in l:
    train[x].fillna(0,inplace=True)
    test[x].fillna(0,inplace=True)

In [19]:
l = ['D_COUNT_L3_MON_01','D_COUNT_L3_MON_02','D_COUNT_L3_MON_03','D_COUNT_L3_MON_04','D_COUNT_L3_MON_05','D_COUNT_L3_MON_06']

In [20]:
for x in l:
    train[x].fillna(0,inplace=True)
    test[x].fillna(0,inplace=True)

In [32]:
train.columns[train.isnull().any()].tolist()

['CR_LIMIT']

In [28]:
train['EOP_MON_03'].fillna(0,inplace=True)

In [29]:
train['EOP_MON_02'].fillna(0,inplace=True)

In [30]:
test['EOP_MON_03'].fillna(0,inplace=True)

In [31]:
test['EOP_MON_02'].fillna(0,inplace=True)

In [36]:
train['CR_LIMIT'].fillna(0,inplace=True)

In [38]:
test['CR_LIMIT'].fillna(0,inplace=True)

In [41]:
test.columns[test.isnull().any()].tolist()

[]

In [40]:
test.fillna(0,inplace=True)

In [42]:
test['CRED_NEED_SCORE']= test['CRED_NEED_SCORE']*100

In [46]:
train.to_csv('train_modi2.csv',header=True,index=False)

In [45]:
test.to_csv('test_modi2.csv',header=True,index=False)

In [48]:
train=pd.read_csv('train_modi2.csv')

In [49]:
test = pd.read_csv('test_modi2.csv')

In [65]:
t = pd.read_csv('test_plBmD8c.csv')

C:\ProgramData\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (8,11,12,13,14,15,16,17,18,19,26,28,29,30,82,83,85,91,269,325,326,327,328) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [52]:
# importing useful libs on the go
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import KFold
import math
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
from sklearn import cross_validation
from sklearn import tree
from sklearn import svm
from sklearn import ensemble
from sklearn import neighbors
from sklearn import linear_model
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
import collections
from sklearn.grid_search import GridSearchCV

plt.style.use('fivethirtyeight') # Good looking plots
pd.set_option('display.max_columns', None) # Display any number of columns

% matplotlib inline

C:\ProgramData\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\ProgramData\Anaconda2\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [50]:
X1 = train.drop('RESPONDERS',axis=1)
y1 = train['RESPONDERS']

In [53]:
scale= StandardScaler()
X1 = scale.fit_transform(X1)

In [54]:
label_encoder = LabelEncoder()
y1 = label_encoder.fit_transform(y1)

In [57]:
# dtrain = xgb.DMatrix(X_train, label=y_train)
# dtest = xgb.DMatrix(X_test, label=y_test)

dtrain = xgb.DMatrix(X1, label=y1)
dtest = xgb.DMatrix(scale.fit_transform(test))

In [279]:
# params2 = {
#     # Parameters that we are going to tune.
#     'max_depth':6,
#     'min_child_weight': 1,
#     'eta':.3,
#     'subsample': 1,
#     'colsample_bytree': 1,
#     # Other parameters
#     'objective':'multi:softprob',
#     'num_class':2
# }

# n_rounds = 15

parameters = { #when use hyperthread, xgboost may become slower
              'objective':['binary:logistic'],
              'learning_rate': [0.08,.5,.1,.3], #so called `eta` value
              'max_depth': [3,5,20],
              'min_child_weight': [2,4],
              'silent': [0],
              'subsample': [0.8,.9,0.7],
              'colsample_bytree': [0.7,.8,.9],
              'n_estimators': [5,10,20], #number of trees, change it to 1000 for better results
              'seed': [1337]}

In [280]:
xgb_model = xgb.XGBClassifier()

In [281]:
clf = GridSearchCV(xgb_model, parameters, n_jobs=-1, 
                   cv=StratifiedKFold(y1, n_folds=5, shuffle=True), 
                   scoring='accuracy',
                   verbose=2, refit=True)


In [282]:
clf.fit(X1,y1)

Fitting 5 folds for each of 648 candidates, totalling 3240 fits


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 13.0min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 52.8min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed: 103.0min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed: 162.5min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed: 253.2min
[Parallel(n_jobs=-1)]: Done 1977 tasks      | elapsed: 361.1min
[Parallel(n_jobs=-1)]: Done 2584 tasks      | elapsed: 483.3min
[Parallel(n_jobs=-1)]: Done 3240 out of 3240 | elapsed: 651.5min finished
C:\ProgramData\Anaconda2\lib\site-packages\xgboost-0.6-py2.7.egg\xgboost\sklearn.py:203: DeprecationWarning: The seed parameter is deprecated as of version .6.Please use random_state instead.seed is deprecated.
  'seed is deprecated.', DeprecationWarning)


GridSearchCV(cv=sklearn.cross_validation.StratifiedKFold(labels=[0 0 ..., 0 0], n_folds=5, shuffle=True, random_state=None),
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [5, 10, 20], 'subsample': [0.8, 0.9, 0.7], 'seed': [1337], 'colsample_bytree': [0.7, 0.8, 0.9], 'silent': [0], 'objective': ['binary:logistic'], 'learning_rate': [0.08, 0.5, 0.1, 0.3], 'max_depth': [3, 5, 20], 'min_child_weight': [2, 4]},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=2)

In [283]:
best_parameters, score, _ = max(clf.grid_scores_, key=lambda x: x[1])
print('Raw AUC score:', score)
for param_name in sorted(best_parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))

('Raw AUC score:', 0.9848633333333333)
colsample_bytree: 0.9
learning_rate: 0.08
max_depth: 20
min_child_weight: 4
n_estimators: 20
objective: 'binary:logistic'
seed: 1337
silent: 0
subsample: 0.8


In [311]:
para ={
    'colsample_bytree': 0.9,
    'learning_rate': 0.07,
    'max_depth': 20,
    'min_child_weight': 4,
    'n_estimators': 20,
    'n_jobs': -1,
    'objective': 'binary:logistic',
    'seed': 1337,
    'silent': 0,
    'subsample': 0.8,
    }



In [312]:
n_rounds=15

In [313]:
bst = xgb.train(para, dtrain,n_rounds)

In [317]:
pred = (bst.predict(dtest) > .2).astype('int')

In [318]:
pred

array([0, 0, 0, ..., 0, 0, 1])

In [319]:
collections.Counter(pred)

Counter({0: 184079, 1: 15921})

In [320]:
t['RESPONDERS'] = pred

In [321]:
ans = t[['CUSTOMER_ID','RESPONDERS']]

In [322]:
ans.to_csv('HOUND2.csv',index=False)